# Exploratory Data Analysis

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 8)
sns.set_theme(context="notebook", style="whitegrid")

## Config

In [ ]:
# file paths
DATA_DIR = Path("../input/tabular-playground-series-aug-2021")

# data
TRAIN_DATA = DATA_DIR / "train.csv"

# columns in the data
INDEX_COL = "id"

TARGET_COL = "loss"

# random state
RANDOM_STATE = 42

## Loading the data

In [ ]:
df = pd.read_csv(TRAIN_DATA, index_col=INDEX_COL)
df.info()

## Separate features from target

In [ ]:
y = df[TARGET_COL]
X = df.drop(TARGET_COL, axis=1)

numeric_dtypes = ["int64", "float64"]
categorical_df = X.select_dtypes(exclude=numeric_dtypes)
numeric_df = X.select_dtypes(include=numeric_dtypes)

## Distribution of variables

### Target variable

In [ ]:
fig, ax = plt.subplots()

sns.histplot(y, ax=ax)
ax.set_title(f"Distribution of {TARGET_COL}")
plt.show()

### Numeric features

In [ ]:
numeric_cols = numeric_df.columns.tolist()
rows = np.ceil(len(numeric_cols) / 2).astype(int)
fig, axes = plt.subplots(rows, 2, figsize=(14, 8 // 2 * rows))
plt.tight_layout()

for i, col in enumerate(numeric_cols):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=df, x=col, ax=ax)
    ax.set_title(f"Histogram of {col}", y=0.94)

plt.show()

### Numeric features by target

In [ ]:
fig, axes = plt.subplots(rows, 2, figsize=(14, 8 // 2 * rows))
plt.tight_layout()

for i, col in enumerate(numeric_cols):
    ax = axes[i // 2, i % 2]
    sns.scatterplot(data=df, x=col, y=TARGET_COL, ax=ax)
    ax.set_title(f"Scatterplot of {col} by {TARGET_COL}", y=0.88)

plt.show()

## Mutual information

In [ ]:
# sampling
y_sample = y.sample(frac=0.2, random_state=RANDOM_STATE)
X_sample = X[X.index.isin(y_sample.index)]

In [ ]:
from sklearn.feature_selection import mutual_info_regression

mi_scores = mutual_info_regression(X_sample, y_sample, random_state=RANDOM_STATE)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(14, len(X.columns)//3))

sns.barplot(x=mi_scores.values, y=mi_scores.index, ax=ax, color="tab:blue")
ax.set_title("Mutual information scores")
plt.show()

In [ ]:
mutual_info = mi_scores[mi_scores > 0]
print("Informative features: {}".format(mutual_info.index.to_list()))